In [6]:
import torch
from torchvision import transforms
from PIL import Image
import requests
from io import BytesIO
import json

import torch.nn as nn
from efficientnet_pytorch import EfficientNet

import os
import random
import pandas as pd
from openpyxl import load_workbook
from tqdm import tqdm

caminho_pasta = 'excels'


analise = []

In [2]:
def selecionar_valores_aleatorios(valores, n):
    valores_filtrados = [
        valor for valor in valores 
        if "americanas" not in str(valor).lower() 
        and "casasbahia" not in str(valor).lower()
        and "imgs.extra" not in str(valor).lower()
        and "imgs.pontofrio" not in str(valor).lower()
    ]
    if len(valores_filtrados) >= n:
        return random.sample(valores_filtrados, n)
    else:
        return valores_filtrados

In [3]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
with open('./class.json', 'r', encoding='utf-8') as arquivo:
    class_names = json.load(arquivo)

# Função para processar a imagem
def process_image(image):
    preprocess = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
    image = preprocess(image).unsqueeze(0)  # Adiciona uma dimensão para o batch
    return image

# Função para fazer a predição
def predict(image_url, model, class_names):
    response = requests.get(image_url)
    image = Image.open(BytesIO(response.content)).convert('RGB')
    image = process_image(image)
    image = image.to(device)

    model.eval()
    with torch.no_grad():
        outputs = model(image)
        _, preds = torch.max(outputs, 1)
        predicted_class = class_names[preds[0]]
    
    return predicted_class

# Caminho para o modelo salvo
model_save_path = 'efficientnet_model.pth'

# Carregar o modelo e as classes
model = EfficientNet.from_name('efficientnet-b0')
num_ftrs = model._fc.in_features
model._fc = nn.Linear(num_ftrs, len(class_names))
model.load_state_dict(torch.load(model_save_path))
model = model.to(device)

In [7]:
for nome_arquivo in tqdm(os.listdir(caminho_pasta)):
    if nome_arquivo.endswith('.xlsx'):
        caminho_arquivo = os.path.join(caminho_pasta, nome_arquivo)
        
        # Abrir e ler o arquivo Excel
        wb = load_workbook(caminho_arquivo)
        ws = wb.active

        # Pegar todos os valores das células (ignorando cabeçalhos se houver)
        valores = []
        for row in ws.iter_rows(min_row=2, values_only=True):  # Ajuste min_row conforme necessário
            valores.extend(row)

        # Filtrar valores não nulos
        valores = [valor for valor in valores if valor is not None]

        # Pegar 4 valores aleatórios que não contenham "americanas"
        valores_aleatorios = selecionar_valores_aleatorios(valores, 4)
        
        cat_arquivo = nome_arquivo.replace('.xlsx','')
        # Criar um dicionário separado para cada valor e adicionar à lista `analise`
        for valor in valores_aleatorios:
            try:
                categoria_predida  = predict(valor, model, class_names)
                comparativo = "errou"
                if categoria_predida == cat_arquivo:
                    comparativo = "acertou"
                    
                resultado = {
                    "arquivo": cat_arquivo,
                    "valor": valor,
                    "predicao":categoria_predida,
                    "status":comparativo
                }
                analise.append(resultado)
            except:
                continue

# Criar um DataFrame a partir da lista `analise`
df = pd.DataFrame(analise)


df.to_excel('analise.xlsx')

print("Análise concluída e salva em 'analise.xlsx'.")

100%|█████████████████████████████████████████| 123/123 [03:59<00:00,  1.94s/it]

Análise concluída e salva em 'analise.csv'.


In [14]:
contagem = df['status'].value_counts()
porcentagem = contagem / contagem.sum() * 100
print('Soma por status --------------')
print(contagem)
print('\nPorcentagem --------------')
print(porcentagem)

Soma por status --------------
status
acertou    412
errou       78
Name: count, dtype: int64

Porcentagem --------------
status
acertou    84.081633
errou      15.918367
Name: count, dtype: float64
